# Part 1

In [1]:
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import json

### Poor Pavement
- Get the pavement in a poor condition dataset

In [2]:
# Define the function to get datesets
def download_geojson(url):
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return pd.json_normalize(data['features'])

def download_json(url):
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame(data)

In [4]:
url = "https://data.cityofnewyork.us/resource/6yyb-pb25.geojson"
RatingLaye = ["POOR","FAIR"] 

def fetch_data(RatingLaye):
    params = {
        "RatingLaye": RatingLaye,
        "$limit": 999999,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        poorpavement_data = pd.DataFrame(data['features'])
        return poorpavement_data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

data_frames = []
for rating in RatingLaye:
    fetched_data = fetch_data(rating)
    if fetched_data is not None:
        data_frames.append(fetched_data)

# Combine the data
poorpavement_data = pd.concat(data_frames, ignore_index=True)
print(poorpavement_data)

          type                                           geometry   
0      Feature  {'type': 'MultiLineString', 'coordinates': [[[...  \
1      Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
2      Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
3      Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
4      Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
...        ...                                                ...   
59899  Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
59900  Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
59901  Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
59902  Feature  {'type': 'MultiLineString', 'coordinates': [[[...   
59903  Feature  {'type': 'MultiLineString', 'coordinates': [[[...   

                                              properties  
0      {'tostreetna': 'BEACH 36 STREET', 'oftcode': '...  
1      {'tostreetna': 'ROCKAWAY FREEWAY', 'oftcode': 

### Vacant Buildings
- Building - Vacant, Open, Unsecured (based on 311 Service Requests)

In [5]:
url = "https://data.cityofnewyork.us/resource/br7h-6m8v.geojson"
start_date = "2022-04-01T00:00:00.000"

params = {
    "$where": f"created_date > '{start_date}'",
    "$limit": 999999,
    "$order": "created_date"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    vacant_building_data = pd.DataFrame(data['features'])
    print(vacant_building_data)
else:
    print(f"Error: {response.status_code} - {response.text}")

         type                                           geometry   
0     Feature  {'type': 'Point', 'coordinates': [-73.83471311...  \
1     Feature  {'type': 'Point', 'coordinates': [-73.87844349...   
2     Feature  {'type': 'Point', 'coordinates': [-73.85896872...   
3     Feature  {'type': 'Point', 'coordinates': [-74.07647500...   
4     Feature  {'type': 'Point', 'coordinates': [-73.94013201...   
...       ...                                                ...   
1426  Feature  {'type': 'Point', 'coordinates': [-74.07988610...   
1427  Feature  {'type': 'Point', 'coordinates': [-73.85113408...   
1428  Feature  {'type': 'Point', 'coordinates': [-73.95737109...   
1429  Feature  {'type': 'Point', 'coordinates': [-74.10300928...   
1430  Feature  {'type': 'Point', 'coordinates': [-73.86418205...   

                                             properties  
0     {'location_state': '', 'facility_type': None, ...  
1     {'location_state': '', 'facility_type': None, ...  
2     {'l

### Facilities

In [6]:
url = "https://data.cityofnewyork.us/resource/ji82-xba5.json"

params = {
    "$limit": 999999
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    facilities_data = pd.DataFrame(data)
    print(facilities_data)
else:
    print(f"Error: {response.status_code} - {response.text}")

                                    uid   
0      9404fa2cf42facc7a9c369f2cf138e2d  \
1      ac9f138b8bdc5a872b0e0c7563c9e6c9   
2      05667e7575d0f5c935dfd7b8aa51c2a7   
3      065dc1e7839e0679eb0a8a402c283b39   
4      07fd286ef3fb329a00c3e35b9069741b   
...                                 ...   
32432  ffedbdbfc952f1906b3fe289b1138b5f   
32433  ffeee9085f9f2e0499a32fdb741df8dc   
32434  fff65938d969a7e681c85648196cc996   
32435  fffdcf59cb9c2f2fe43e94e9d8e81555   
32436  fffec10f6cc9a504a022db6e4fdd41fe   

                                          facname      city      boro   
0                        SEARCH AND RESTORE, INC.  BROOKLYN  BROOKLYN  \
1                                  ARTS INDONESIA  BROOKLYN  BROOKLYN   
2                            P.S. 160 WALT DISNEY     BRONX     BRONX   
3                                       CONCOURSE     BRONX     BRONX   
4                                       OYE GROUP  BROOKLYN  BROOKLYN   
...                                          

### Women's Resources Network

In [7]:
women_resources_data = download_json('https://data.cityofnewyork.us/resource/pqg4-dm6b.json')

### Subway Stations

In [8]:
subway_stations_data = gpd.read_file('/Users/hbliu/Desktop/2023Spring/MUSA509_Cloud_Computing/final_project/Subway Stations/geo_export_eb628050-3b8d-40a4-81fd-5c0e7bdfd694.shp')

### Damaged Streetlight

In [9]:
url = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json"
complaint_type = "Street Light Condition"
start_date = "2022-04-01T00:00:00.000"

params = {
    "complaint_type": complaint_type,
    "$where": f"created_date > '{start_date}'",
    "$limit": 999999,
    "$order": "created_date"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    streetlight_data = pd.DataFrame(data)
    print(streetlight_data)
else:
    print(f"Error: {response.status_code} - {response.text}")

      unique_key             created_date              closed_date agency   
0       53794682  2022-04-01T00:59:00.000  2022-05-19T07:47:00.000    DOT  \
1       53800112  2022-04-01T04:49:00.000  2022-04-06T21:10:00.000    DOT   
2       53805744  2022-04-01T06:00:00.000  2022-04-01T14:40:00.000    DOT   
3       53806864  2022-04-01T07:35:00.000  2022-04-04T14:00:00.000    DOT   
4       53802394  2022-04-01T07:41:00.000  2022-04-04T12:51:00.000    DOT   
...          ...                      ...                      ...    ...   
58920   57426081  2023-04-26T21:15:00.000                      NaN    DOT   
58921   57428555  2023-04-26T22:46:00.000                      NaN    DOT   
58922   57428736  2023-04-26T22:48:00.000                      NaN    DOT   
58923   57426356  2023-04-26T23:01:00.000                      NaN    DOT   
58924   57427283  2023-04-27T00:16:00.000                      NaN    DOT   

                        agency_name          complaint_type   
0      Depar

In [10]:
from google.cloud import bigquery

In [13]:
import os
import pyarrow

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/hbliu/Desktop/2023Spring/MUSA509_Cloud_Computing/final_project/musa509-final-project-5c7782a87a33.json'


In [14]:
def create_dataset(dataset_name):
    client = bigquery.Client()
    dataset_id = f"{client.project}.{dataset_name}"
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    dataset = client.create_dataset(dataset, exists_ok=True)  # Set exists_ok=True to avoid an error if the dataset already exists.
    print(f"Created dataset {client.project}.{dataset.dataset_id}")

create_dataset('infrastructure_dataset')


Created dataset musa509-final-project.infrastructure_dataset


In [15]:
def flatten_coordinates(coordinates):
    if len(coordinates) > 0 and isinstance(coordinates[0], list):
        return coordinates[0][0]
    else:
        return None

poorpavement_data['geometry_coordinates'] = poorpavement_data.apply(flatten_coordinates, axis=1)
facilities_data['geometry_coordinates'] = facilities_data.apply(lambda row: [row['longitude'], row['latitude']], axis=1)
women_resources_data['geometry_coordinates'] = women_resources_data.apply(lambda row: [row['longitude'], row['latitude']], axis=1)

poorpavement_data.rename(columns=lambda x: x.replace(".", "_"), inplace=True)
streetlight_data.rename(columns=lambda x: x.replace(".", "_"), inplace=True)
vacant_building_data.rename(columns=lambda x: x.replace(".", "_"), inplace=True)
facilities_data.rename(columns=lambda x: x.replace(".", "_"), inplace=True)
women_resources_data.rename(columns=lambda x: x.replace(".", "_"), inplace=True)
subway_stations_data.rename(columns=lambda x: x.replace(".", "_"), inplace=True)

women_resources_data['geometry_coordinates'] = women_resources_data['geometry_coordinates'].apply(lambda x: np.nan if x == [np.nan, np.nan] else x)
women_resources_data.dropna(subset=['geometry_coordinates'], inplace=True)

subway_stations_data['x'] = subway_stations_data['geometry'].apply(lambda geom: geom.x)
subway_stations_data['y'] = subway_stations_data['geometry'].apply(lambda geom: geom.y)
subway_stations_data['geometry_coordinates'] = subway_stations_data['geometry'].apply(lambda geom: (geom.x, geom.y))

import json

facilities_data['geometry_coordinates'] = facilities_data['geometry_coordinates'].apply(json.dumps)
women_resources_data['geometry_coordinates'] = women_resources_data['geometry_coordinates'].apply(json.dumps)

# Drop the geometry column for poorpavement_data DataFrame
poorpavement_data.drop(columns=['geometry'], inplace=True)

def load_data_to_bigquery(dataset_name, table_name, dataframe):
    client = bigquery.Client()
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.autodetect = True

    load_job = client.load_table_from_dataframe(dataframe, table_ref, job_config=job_config)
    load_job.result()

# Load datasets into BigQuery
load_data_to_bigquery('infrastructure_dataset', 'pavement', poorpavement_data)
load_data_to_bigquery('infrastructure_dataset', 'streetlight', streetlight_data)
load_data_to_bigquery('infrastructure_dataset', 'vacant_building', vacant_building_data)
load_data_to_bigquery('infrastructure_dataset', 'facilities', facilities_data)
load_data_to_bigquery('infrastructure_dataset', 'women_resources', women_resources_data)
load_data_to_bigquery('infrastructure_dataset', 'subway_stations', subway_stations_data)


/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/google/cloud/bigquery/_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: properties, geometry_coordinates.
  warnings.warn(
/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/google/cloud/bigquery/_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: location.
  warnings.warn(
/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/google/cloud/bigquery/_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: geometry, properties.
  warnings.warn(
/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/google/cloud/bigquery/_pandas_helpers.py:603: UserWarning: Pyarrow could not determine the type of columns: location_1.
  warnings.warn(


# Part 2

In [16]:
from google.cloud import bigquery


def run_query(query):
    client = bigquery.Client()
    query_job = client.query(query)
    results = query_job.result()
    return results.to_dataframe()

In [17]:
def get_table_count(dataset_name, table_name):
    query = f"""
    SELECT COUNT(*)
    FROM `{"musa509-final-project"}.{dataset_name}.{table_name}`
    """
    count_dataframe = run_query(query)
    return count_dataframe.iloc[0, 0]

table_names = [
    "pavement",
    "streetlight",
    "vacant_building",
    "facilities",
    "women_resources",
    "subway_stations",
]

dataset_name = "infrastructure_dataset"

for table_name in table_names:
    count = get_table_count(dataset_name, table_name)
    print(f"Table {table_name} has {count} records.")


Table pavement has 59904 records.
Table streetlight has 58925 records.
Table vacant_building has 1431 records.
Table facilities has 32437 records.
Table women_resources has 680 records.
Table subway_stations has 473 records.


# Part 3
- aggregate features into census tract dataset

- New York City consists of five counties:
    - Bronx County (code: 005)
    - Kings County (Brooklyn) (code: 047)
    - New York County (Manhattan) (code: 061)
    - Queens County (code: 081)
    - Richmond County (Staten Island) (code: 085)


In [18]:
os.environ["CENSUS_API_KEY"] = "a9e713a06a0a0f8ec8531e047c9d01e7d9f507d9"
dataset = "acs/acs5"
year = "2021"

variables = ["B01003_001E", "B25077_001E", "B25058_001E", "B17001_002E", "B19013_001E"]
variables_str = ",".join(variables)
url = f"https://api.census.gov/data/{year}/{dataset}?get={variables_str}&for=tract:*&in=state:36%20county:005,047,061,081,085&key={os.environ['CENSUS_API_KEY']}"

response = requests.get(url)
data = response.json()
header, *rows = data

In [19]:
create_dataset('census_dataset')

census_data = pd.DataFrame(rows, columns=header)
census_data = census_data.astype({var: "float64" for var in variables})

Created dataset musa509-final-project.census_dataset


In [20]:
shapefile_path = "/Users/hbliu/Desktop/2023Spring/MUSA509_Cloud_Computing/final_project/tl_2021_36_tract/tl_2021_36_tract.shp"
gdf = gpd.read_file(shapefile_path)

# Filter for the required counties
nyc_county_fips = ['005', '047', '061', '081', '085']
gdf = gdf[gdf['COUNTYFP'].isin(nyc_county_fips)]

# Rename columns for BigQuery
gdf.rename(columns={"TRACTCE": "tract_id", "geometry": "tract_geometry"}, inplace=True)

# Simplify the geometry to reduce the size
gdf["tract_geometry"] = gdf["tract_geometry"].simplify(0.0001, preserve_topology=True)


In [21]:
import geopandas as gpd
from shapely.geometry import shape, mapping

# Assuming you have a GeoDataFrame called 'census_data'
def fix_geometry(geom):
    shapely_geom = shape(geom)
    corrected_geom = shapely_geom.buffer(0)
    return mapping(corrected_geom)

gdf["tract_geometry"] = gdf["tract_geometry"].apply(fix_geometry)


In [23]:
census_data = gdf.merge(census_data, left_on="tract_id", right_on="tract")

In [24]:
census_data = census_data.rename(columns={'B01003_001E': 'total_population', 
                                          'B25077_001E': 'median_house_value',
                                          'B25058_001E':'median_contract_rent',
                                          'B17001_002E':'people_below_poverty',
                                          'B19013_001E':'median_household_income',
                                          'tract_geometry':'geometry'})

In [25]:
census_data['poverty_rate'] = census_data['people_below_poverty']/census_data['total_population']
census_data['geometry'] = census_data['geometry'].apply(json.dumps)

census_data.head()

,STATEFP,COUNTYFP,tract_id,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,geometry,total_population,median_house_value,median_contract_rent,people_below_poverty,median_household_income,state,county,tract,poverty_rate
0,36,047,069601,36047069601,696.01,Census Tract 696.01,G5020,S,456923,47776,...,"{""type"": ""Polygon"", ""coordinates"": [[[-73.9186...",3400.0,685800.0,1825.0,174.0,86023.0,36,047,069601,0.051176
1,36,047,069602,36047069602,696.02,Census Tract 696.02,G5020,S,688168,172866,...,"{""type"": ""Polygon"", ""coordinates"": [[[-73.9174...",5748.0,705800.0,1355.0,507.0,84619.0,36,047,069602,0.088205
2,36,047,079801,36047079801,798.01,Census Tract 798.01,G5020,S,129680,0,...,"{""type"": ""Polygon"", ""coordinates"": [[[-73.9607...",3198.0,706400.0,1626.0,91.0,71583.0,36,047,079801,0.028455
3,36,047,079802,36047079802,798.02,Census Tract 798.02,G5020,S,106366,0,...,"{""type"": ""Polygon"", ""coordinates"": [[[-73.9628...",5129.0,890600.0,1704.0,686.0,84426.0,36,047,079802,0.133749
4,36,047,105801,36047105801,1058.01,Census Tract 1058.01,G5020,S,418443,0,...,"{""type"": ""Polygon"", ""coordinates"": [[[-73.8855...",6094.0,-666666666.0,536.0,2206.0,19826.0,36,047,105801,0.361995


In [26]:
load_data_to_bigquery('census_dataset', 'nyc_census', census_data)

- Data Wrangling

In [27]:
from shapely.geometry import shape

census_data['geometry'] = census_data['geometry'].apply(lambda x: shape(json.loads(x)))
census_data = gpd.GeoDataFrame(census_data, geometry='geometry')

In [28]:
census_data.crs = "EPSG:4326"
print(census_data.crs)
print(subway_stations_data.crs)

EPSG:4326
GEOGCS["WGS84(DD)",DATUM["WGS84",SPHEROID["WGS84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]


In [30]:
from geopandas import sjoin
from rtree import index

subway_stations_data['index'] = subway_stations_data.index
spatial_index = subway_stations_data.sindex

In [31]:
def count_within_buffer(tract_geometry, buffer_radius, stations_data, spatial_index):
    possible_matches_index = list(spatial_index.intersection(tract_geometry.buffer(buffer_radius).bounds))
    possible_matches = stations_data.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(tract_geometry.buffer(buffer_radius))]
    return len(precise_matches)

In [32]:
from shapely.geometry import Point

poorpavement_data['geometry'] = poorpavement_data['geometry_coordinates'].apply(lambda coord: Point(coord) if coord is not None else None)
poorpavement_data = gpd.GeoDataFrame(poorpavement_data, geometry='geometry')

streetlight_data['geometry'] = streetlight_data.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
streetlight_data = gpd.GeoDataFrame(streetlight_data, geometry='geometry')

vacant_building_data['longitude'] = vacant_building_data['properties'].apply(lambda row: row['longitude'])
vacant_building_data['latitude'] = vacant_building_data['properties'].apply(lambda row: row['latitude'])

In [33]:
vacant_building_data['longitude'] = vacant_building_data['longitude'].fillna(0)
vacant_building_data['latitude'] = vacant_building_data['latitude'].fillna(0)
vacant_building_data['geometry'] = vacant_building_data.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)
vacant_building_data = vacant_building_data.dropna(subset=['longitude', 'latitude'])
vacant_building_data = gpd.GeoDataFrame(vacant_building_data, geometry='geometry')

In [34]:
facilities_data['geometry'] = facilities_data.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)
facilities_data = gpd.GeoDataFrame(facilities_data, geometry='geometry')

In [35]:
women_resources_data['geometry'] = gpd.points_from_xy(women_resources_data['longitude'], women_resources_data['latitude'])
women_resources_data = gpd.GeoDataFrame(women_resources_data, geometry='geometry')

In [36]:
poorpavement_data['index'] = poorpavement_data.index
streetlight_data['index'] = streetlight_data.index
vacant_building_data['index'] = vacant_building_data.index
facilities_data['index'] = facilities_data.index
women_resources_data['index'] = women_resources_data.index

spatial_index_pavement = poorpavement_data.sindex
spatial_index_streetlight = streetlight_data.sindex
spatial_index_vacant_building = vacant_building_data.sindex
spatial_index_facilities = facilities_data.sindex
spatial_index_women_resources = women_resources_data.sindex

In [37]:
buffer_radius = 0.01

census_data['subway_stations_count'] = census_data['geometry'].apply(
    lambda x: count_within_buffer(x, buffer_radius, subway_stations_data, spatial_index)
)

In [38]:

census_data['poorpavement_count'] = census_data['geometry'].apply(
    lambda x: count_within_buffer(x, buffer_radius, poorpavement_data, spatial_index_pavement)
)

In [39]:

census_data['streetlight_count'] = census_data['geometry'].apply(
    lambda x: count_within_buffer(x, buffer_radius, streetlight_data, spatial_index_streetlight)
)

In [40]:

census_data['vacant_buildings_count'] = census_data['geometry'].apply(
    lambda x: count_within_buffer(x, buffer_radius, vacant_building_data, spatial_index_vacant_building)
)

In [41]:

census_data['facilities_count'] = census_data['geometry'].apply(
    lambda x: count_within_buffer(x, buffer_radius, facilities_data, spatial_index_facilities)
)


In [42]:
census_data['women_resources_count'] = census_data['geometry'].apply(
    lambda x: count_within_buffer(x, buffer_radius, women_resources_data, spatial_index_women_resources)
)

In [43]:
census_data

,STATEFP,COUNTYFP,tract_id,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,state,county,tract,poverty_rate,subway_stations_count,poorpavement_count,streetlight_count,vacant_buildings_count,facilities_count,women_resources_count
0,36,047,069601,36047069601,696.01,Census Tract 696.01,G5020,S,456923,47776,...,36,047,069601,0.051176,0,0,215,13,149,0
1,36,047,069602,36047069602,696.02,Census Tract 696.02,G5020,S,688168,172866,...,36,047,069602,0.088205,0,0,192,12,110,0
2,36,047,079801,36047079801,798.01,Census Tract 798.01,G5020,S,129680,0,...,36,047,079801,0.028455,8,0,381,12,316,5
3,36,047,079802,36047079802,798.02,Census Tract 798.02,G5020,S,106366,0,...,36,047,079802,0.133749,5,0,328,9,269,5
4,36,047,105801,36047105801,1058.01,Census Tract 1058.01,G5020,S,418443,0,...,36,047,105801,0.361995,0,0,283,4,160,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4620,36,005,022902,36005022902,229.02,Census Tract 229.02,G5020,S,91601,0,...,36,005,022902,0.349865,7,0,944,7,545,9
4621,36,005,045101,36005045101,451.01,Census Tract 451.01,G5020,S,176003,0,...,36,005,045101,0.024161,3,0,650,3,104,0
4622,36,005,040304,36005040304,403.04,Census Tract 403.04,G5020,S,67355,0,...,36,005,040304,0.307333,6,0,901,7,326,6
4623,36,081,127700,36081127700,1277,Census Tract 1277,G5020,S,1069117,0,...,36,081,127700,0.007205,2,0,378,6,128,2


In [44]:
census_data['geometry'] = census_data['geometry'].to_wkt()

/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [45]:
load_data_to_bigquery('census_dataset', 'nyc_census', census_data)

In [46]:
census_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4625 entries, 0 to 4624
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   STATEFP                  4625 non-null   object 
 1   COUNTYFP                 4625 non-null   object 
 2   tract_id                 4625 non-null   object 
 3   GEOID                    4625 non-null   object 
 4   NAME                     4625 non-null   object 
 5   NAMELSAD                 4625 non-null   object 
 6   MTFCC                    4625 non-null   object 
 7   FUNCSTAT                 4625 non-null   object 
 8   ALAND                    4625 non-null   int64  
 9   AWATER                   4625 non-null   int64  
 10  INTPTLAT                 4625 non-null   object 
 11  INTPTLON                 4625 non-null   object 
 12  geometry                 4625 non-null   object 
 13  total_population         4625 non-null   float64
 14  median_house_val

In [107]:
import os
import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import geopandas as gpd
import pandas as pd


app = dash.Dash(__name__)
save_directory = "/Users/hbliu/Desktop/2023Spring/MUSA509_Cloud_Computing/final_project/"
save_filename = "app.py"

source_code = '''
import os
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import geopandas as gpd
from google.cloud import bigquery
from shapely import wkt

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/hbliu/Desktop/2023Spring/MUSA509_Cloud_Computing/final_project/musa509-final-project-5c7782a87a33.json"

def fetch_census_data():
    # Initialize a BigQuery client
    client = bigquery.Client()
    query = """
        SELECT *
        FROM `musa509-final-project.census_dataset.nyc_census`
    """
    query_job = client.query(query)
    census_data = query_job.to_dataframe()
    return census_data

census_data = fetch_census_data()
census_data["geometry"] = census_data["geometry"].apply(wkt.loads)
census_data = gpd.GeoDataFrame(census_data, geometry="geometry")
census_data.loc[census_data['median_house_value'] < 0, 'median_house_value'] = 0
census_data.loc[census_data['median_household_income'] < 0, 'median_household_income'] = 0

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.index_string = """
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>Infrastructure Overview in NYC</title>
        {%favicon%}
        {%css%}
        <style>
            body {
                font-family: Arial, sans-serif;
                font-size: 16px;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
"""


app.layout = html.Div([
    html.H1("Infrastructure Overview in New York City", 
            style={"textAlign": "center", "font-family": "Arial, sans-serif", "font-size": "26px"}),
    html.Div(children='MUSA 509 | Final Project | Haobing Liu', style={
        'textAlign': 'center',
        'color': 'black'
    }),        
    dbc.Row([
        dbc.Col([
            html.Label("Select a feature:"),
            dcc.Dropdown(
                id="feature_dropdown",
                options=[
                    {"label": "Number of Subway Stations per tract", "value": "subway_stations_count"},
                    {"label": "Number of Damaged Streetlight per tract", "value": "streetlight_count"},
                    {"label": "Number of Vacant Buildings per tract", "value": "vacant_buildings_count"},
                    {"label": "Number of Facilities per tract", "value": "facilities_count"},
                    {"label": "Number of Women's Resources per tract", "value": "women_resources_count"},
                    {"label": "Total Population", "value": "total_population"},
                    {"label": "Median House Value per tract", "value": "median_house_value"},
                    {"label": "Median Household Income per tract", "value": "median_household_income"},
                    {"label": "Poverty Rate per tract", "value": "poverty_rate"},
                ],
                value="streetlight_count",
            ),
        ]),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id="choropleth_map", className="full-height full-width")
        ], className="full-height"),
    ], className="full-height",style={"backgroundColor": "#262626"}),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id="histogram", className="full-height")
        ],width=5, style={"height": "100vh"}),
    ]),
], style={"height": "100vh"})


@app.callback(
    Output("choropleth_map", "figure"),
    [Input("feature_dropdown", "value")],
)

def update_choropleth_map(feature):
    fig = px.choropleth_mapbox(
        census_data,
        geojson=census_data.geometry,
        locations=census_data.index,
        color=feature,
        color_continuous_scale=["#fffef0","#c8fa52","#c9ffea", "#8fffd2", "#42f5ad"],
        mapbox_style="carto-darkmatter",
        zoom=9,
        center={"lat": 40.7128, "lon": -74.0060},
        opacity=0.7,
        hover_data=["NAMELSAD"],
        labels={feature: f"{feature.replace('_', ' ').capitalize()}"},
    )

    fig.update_layout(
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    title=f"{feature.replace('_', ' ').capitalize()} in NYC",
    plot_bgcolor="#262626", 
    paper_bgcolor="#262626",
    font=dict(color="#FFFFFF")
    )
    fig.update_traces(
    marker=dict(line=dict(color="white", width=0.2)))

    return fig
    
@app.callback(
    Output("histogram", "figure"),
    [Input("feature_dropdown", "value")],
)

def update_histogram(feature):
    fig = px.histogram(census_data, x=feature, nbins=30)
    fig.update_layout(
    title=f"{feature.replace('_', ' ').capitalize()} Distribution",
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font=dict(color="#262626"),
     xaxis=dict(
            showgrid=False,
            showticklabels=True,
            zeroline=False,
            ticks='outside',
            linewidth=1,
            linecolor='white',
        ),
        yaxis=dict(
            showgrid=False,
            showticklabels=True,
            zeroline=False,
            ticks='outside',
            linewidth=1,
            linecolor='white',
        )
    )
    fig.update_traces(
    marker=dict(
    line=dict(color="#3d403f", width=3),
    color="#c8fa52"
        )
        )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, port=8051)
'''

with open(os.path.join(save_directory, save_filename), "w") as f:
    f.write(source_code)

if __name__ == "__main__":
    app.run_server(debug=True, port=8058)

Dash is running on http://127.0.0.1:8058/

 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 678, in initialize
    self.init_sockets()
  File "/Users/hbliu/anaconda3/envs/musa509/lib/python3.10/site-packages/

SystemExit: 1